# Tracing for Different Types of Runs

### Types of Runs

LangSmith supports many different types of Runs - you can specify what type your Run is in the @traceable decorator. The types of runs are:

- LLM: Invokes an LLM
- Retriever: Retrieves documents from databases or other sources
- Tool: Executes actions with function calls
- Chain: Default type; combines multiple Runs into a larger process
- Prompt: Hydrates a prompt to be used with an LLM
- Parser: Extracts structured data

### Setup

In [7]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

In [32]:
from langsmith import traceable

inputs = [
  {"role": "system", "content": "You are a helpful assistant that knows lot about travel ."},
  {"role": "user", "content": "I'd like to book a table for two."},
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "Sure, what time would you like to book the table for?"
          }
      }
  ]
}

# Can also use one of:
# output = {
#     "message": {
#         "role": "assistant",
#         "content": "Sure, what time would you like to book the table for?"
#     }
# }
#
# output = {
#     "role": "assistant",
#     "content": "Sure, what time would you like to book the table for?"
# }
#
# output = ["assistant", "Sure, what time would you like to book the table for?"]

@traceable(
  # TODO: Add an run_type="llm", and metadata for ls_provider, and ls_model_name
)
def chat_model(messages: list):
  return output

chat_model(inputs)

{'choices': [{'message': {'role': 'assistant',
    'content': 'Sure, what time would you like to book the table for?'}}]}

### LLM Runs for Chat Models

LangSmith provides special rendering and processing for LLM traces. In order to make the most of this feature, you must log your LLM traces in a specific format.

For chat-style models, inputs must be a list of messages in OpenAI-compatible format, represented as Python dictionaries or TypeScript object. Each message must contain the key role and content.

The output is accepted in any of the following formats:

- A dictionary/object that contains the key choices with a value that is a list of dictionaries/objects. Each dictionary/object must contain the key message, which maps to a message object with the keys role and content.
- A dictionary/object that contains the key message with a value that is a message object with the keys role and content.
- A tuple/array of two elements, where the first element is the role and the second element is the content.
- A dictionary/object that contains the key role and content.
The input to your function should be named messages.

You can also provide the following metadata fields to help LangSmith identify the model and calculate costs. If using LangChain or OpenAI wrapper, these fields will be automatically populated correctly.
- ls_provider: The provider of the model, eg "openai", "anthropic", etc.
- ls_model_name: The name of the model, eg "gpt-4o-mini", "claude-3-opus-20240307", etc.

In [ ]:
from langsmith import traceable

inputs = [
  {"role": "system", "content": "You are a helpful assistant, who knows about travel.  Please suggest best travel destination based on user's interest."},
  {"role": "user", "content": "i would like to travel what is the best to travel"},
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "i would like to go to europe suggest good places there."
          }
      }
  ]
}

# Can also use one of:
# output = {
#     "message": {
#         "role": "assistant",
#         "content": "Sure, what time would you like to book the table for?"
#     }
# }
#
# output = {
#     "role": "assistant",
#     "content": "Sure, what time would you like to book the table for?"
# }
#
# output = ["assistant", "Sure, what time would you like to book the table for?"]

@traceable(
  run_type="llm", ls_provider="openai", ls_model_name="gpt-4o-mini"
)
def chat_model(messages: list):
  return output

chat_model(inputs)

{'choices': [{'message': {'role': 'assistant',
    'content': 'i would like to go to europe suggest good places there.'}}]}

### Handling Streaming LLM Runs

For streaming, you can "reduce" the outputs into the same format as the non-streaming version. This is currently only supported in Python.

Liam


### Retriever Runs + Documents

Many LLM applications require looking up documents from vector databases, knowledge graphs, or other types of indexes. Retriever traces are a way to log the documents that are retrieved by the retriever. LangSmith provides special rendering for retrieval steps in traces to make it easier to understand and diagnose retrieval issues. In order for retrieval steps to be rendered correctly, a few small steps need to be taken.

1. Annotate the retriever step with run_type="retriever".
2. Return a list of Python dictionaries or TypeScript objects from the retriever step. Each dictionary should contain the following keys:
    - page_content: The text of the document.
    - type: This should always be "Document".
    - metadata: A python dictionary or TypeScript object containing metadata about the document. This metadata will be displayed in the trace.

In [30]:
from langsmith import traceable

def _convert_docs(results):
  return [
      {
          "page_content": r,
          "type": "Document", # This is the wrong format! The key should be type
          "metadata": {"foo": "bar"}
      }
      for r in results
  ]

@traceable(
    # TODO: Add an run_type="retriever"
)
def retrieve_docs(query):
  # Retriever returning hardcoded dummy documents.
  # In production, this could be a real vector datatabase or other document index.
  contents = ["Document contents 1", "Document contents 2", "Document contents 3"]
  return _convert_docs(contents)

retrieve_docs("User query")
from langsmith import traceable

# Optional: merge streaming chunks into one
def _reduce_chunks(chunks: list):
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={"ls_provider": "my_provider", "ls_model_name": "baby_name_suggester"},
    reduce_fn=_reduce_chunks  # merge all chunks into one output
)
def suggest_baby_names(messages: list):
    # Example name suggestions
    name_suggestions = ["Liam", "Olivia", "Noah", "Emma", "Aiden"]
    
    for name in name_suggestions:
        yield {
            "choices": [
                {
                    "message": {
                        "content": name,
                        "role": "assistant",
                    }
                }
            ]
        }

# User prompt
user_messages = [
    {"role": "system", "content": "You are a helpful assistant that suggests baby names."},
    {"role": "user", "content": "I just had a baby. Can you suggest some names?"}
]

# Run the streaming chat
result = list(suggest_baby_names(user_messages))

# Print final merged output
print(result[0]["choices"][0]["message"]["content"])


Liam


### Tool Calling

LangSmith has custom rendering for Tool Calls made by the model to make it clear when provided tools are being used.

In [15]:
# --- Imports and Environment Setup ---
import os
from dotenv import load_dotenv
from openai import OpenAI
from langsmith import traceable
from typing import List, Optional
import json

# Load API keys from .env
load_dotenv()

# Initialize OpenAI client
openai_client = OpenAI()

# Enable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"


# --- XOR Tool Definition ---
@traceable(run_type="tool")
def XOR(a: int, b: int):
    """Perform XOR operation on two integers"""
    result = a ^ b
    print(f"  [Tool Execution] XOR({a}, {b}) = {result}")
    return result

# --- LLM call with tool definitions ---
@traceable(run_type="llm")
def call_groq_with_tools(messages: list, tools: Optional[list]):
    """Call OpenAI / Groq with tool definitions"""
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        tools=tools,
        max_tokens=200
    )
    return response

# --- Chain orchestrator ---
@traceable(run_type="chain")
def ask_about_xor(inputs: list, tools: list):
    # First call: LLM decides if it needs to use a tool
    print("\n[Step 1] Sending initial request to LLM...")
    response = call_groq_with_tools(inputs, tools)
    
    # Check if model wants to use a tool
    if response.choices[0].message.tool_calls:
        print("[Step 2] LLM decided to use a tool!")
        
        tool_call = response.choices[0].message.tool_calls[0]
        tool_call_args = json.loads(tool_call.function.arguments)
        
        print(f"[Step 3] Tool: {tool_call.function.name}")
        print(f"[Step 4] Arguments: {tool_call_args}")
        
        # Execute the tool
        result = XOR(tool_call_args["a"], tool_call_args["b"])
        
        # Add tool response to conversation
        tool_response_message = {
            "role": "tool",
            "content": json.dumps({
                "a": tool_call_args["a"],
                "b": tool_call_args["b"],
                "result": result,
            }),
            "tool_call_id": tool_call.id
        }
        
        inputs.append(response.choices[0].message)
        inputs.append(tool_response_message)
        
        # Second call: LLM generates final response with tool results
        print("[Step 5] Sending tool results back to LLM for final response...")
        final_response = call_groq_with_tools(inputs, None)
        return final_response
    
    print("[Info] No tool call was made by the LLM")
    return response

# --- Tool specification ---
tools = [
    {
        "type": "function",
        "function": {
            "name": "XOR",
            "description": "Perform XOR (exclusive OR) operation on two integers",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "integer", "description": "First integer to XOR"},
                    "b": {"type": "integer", "description": "Second integer to XOR"},
                },
                "required": ["a", "b"]
            }
        }
    }
]

# --- Inputs ---
xor_inputs = [
    {"role": "system", "content": "You are a helpful assistant that can perform mathematical operations."},
    {"role": "user", "content": "What is the value of 30 XOR 42?"},
]

# --- Run the XOR demo ---
xor_response = ask_about_xor(xor_inputs, tools)
print(f"\nAssistant: {xor_response.choices[0].message.content}")
# --- XOR Tool Definition ---
@traceable(run_type="tool")
def XOR(a: int, b: int):
    """Perform XOR operation on two integers"""
    result = a ^ b
    print(f"  [Tool Execution] XOR({a}, {b}) = {result}")
    return result

# --- LLM call with tool definitions ---
@traceable(run_type="llm")
def call_groq_with_tools(messages: list, tools: Optional[list]):
    """Call OpenAI / Groq with tool definitions"""
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        tools=tools,
        max_tokens=200
    )
    return response

# --- Chain orchestrator ---
@traceable(run_type="chain")
def ask_about_xor(inputs: list, tools: list):
    # First call: LLM decides if it needs to use a tool
    print("\n[Step 1] Sending initial request to LLM...")
    response = call_groq_with_tools(inputs, tools)
    
    # Check if model wants to use a tool
    if response.choices[0].message.tool_calls:
        print("[Step 2] LLM decided to use a tool!")
        
        tool_call = response.choices[0].message.tool_calls[0]
        tool_call_args = json.loads(tool_call.function.arguments)
        
        print(f"[Step 3] Tool: {tool_call.function.name}")
        print(f"[Step 4] Arguments: {tool_call_args}")
        
        # Execute the tool
        result = XOR(tool_call_args["a"], tool_call_args["b"])
        
        # Add tool response to conversation
        tool_response_message = {
            "role": "tool",
            "content": json.dumps({
                "a": tool_call_args["a"],
                "b": tool_call_args["b"],
                "result": result,
            }),
            "tool_call_id": tool_call.id
        }
        
        inputs.append(response.choices[0].message)
        inputs.append(tool_response_message)
        
        # Second call: LLM generates final response with tool results
        print("[Step 5] Sending tool results back to LLM for final response...")
        final_response = call_groq_with_tools(inputs, None)
        return final_response
    
    print("[Info] No tool call was made by the LLM")
    return response

# --- Tool specification ---
tools = [
    {
        "type": "function",
        "function": {
            "name": "XOR",
            "description": "Perform XOR (exclusive OR) operation on two integers",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "integer", "description": "First integer to XOR"},
                    "b": {"type": "integer", "description": "Second integer to XOR"},
                },
                "required": ["a", "b"]
            }
        }
    }
]

# --- Inputs ---
xor_inputs = [
    {"role": "system", "content": "You are a helpful assistant that can perform mathematical operations."},
    {"role": "user", "content": "What is the value of 30 XOR 42?"},
]

# --- Run the XOR demo ---
xor_response = ask_about_xor(xor_inputs, tools)
print(f"\nAssistant: {xor_response.choices[0].message.content}")



[Step 1] Sending initial request to LLM...
[Step 2] LLM decided to use a tool!
[Step 3] Tool: XOR
[Step 4] Arguments: {'a': 30, 'b': 42}
  [Tool Execution] XOR(30, 42) = 52
[Step 5] Sending tool results back to LLM for final response...

Assistant: The value of 30 XOR 42 is 52.

[Step 1] Sending initial request to LLM...
[Step 2] LLM decided to use a tool!
[Step 3] Tool: XOR
[Step 4] Arguments: {'a': 30, 'b': 42}
  [Tool Execution] XOR(30, 42) = 52
[Step 5] Sending tool results back to LLM for final response...

Assistant: The value of 30 XOR 42 is 52.
